In [ ]:
!wget http://cs301.pintoenterprisetech.com/GME.csv

--2021-11-17 20:28:54--  http://cs301.pintoenterprisetech.com/GME.csv
Resolving cs301.pintoenterprisetech.com (cs301.pintoenterprisetech.com)... 157.245.119.43, 2604:a880:800:10::904:6001
Connecting to cs301.pintoenterprisetech.com (cs301.pintoenterprisetech.com)|157.245.119.43|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129689 (127K) [text/csv]
Saving to: ‘GME.csv’

GME.csv             100%[===================>] 126.65K  --.-KB/s    in 0.08s   

2021-11-17 20:28:55 (1.50 MB/s) - ‘GME.csv’ saved [129689/129689]



In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
first_data_collection_date = datetime.datetime.strptime('2021-10-31 04:00:00', "%Y-%m-%d %H:%M:%S")

In [ ]:
df = pd.read_csv("GME.csv")

In [ ]:
GMEdf = df.drop(['timekey', 'ticker'], axis=1)

In [ ]:
GMEdf['timestamp'] = pd.to_datetime(GMEdf['timestamp'])
GMEdf.set_index("timestamp", inplace=True)
GMEdf.head()

,avg(sentiment)
timestamp,
2021-10-31 02:48:20,0.020940
2021-10-31 02:51:21,0.339265
2021-10-31 03:03:52,0.375158
2021-10-31 03:10:07,0.181590
2021-10-31 03:20:05,-0.014369


In [ ]:
date = first_data_collection_date
#Get the sentiment values from the same day, before 4pm into the same row of a dataframe
GMEdf.loc[date+datetime.timedelta(days=1):date+datetime.timedelta(days=2)].reset_index().drop('timestamp', axis=1).T.reset_index().drop('index', axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143
0,0.114356,0.099109,0.175144,0.304779,0.345358,0.399038,-0.011435,0.222067,0.196858,0.172158,0.180473,0.061874,0.026794,0.085606,0.382719,0.298168,0.099157,0.201164,0.19846,0.112562,0.363038,0.255517,0.174993,0.2321,0.20055,0.408267,0.662525,0.523527,0.15031,0.260815,0.178775,0.293928,0.50604,-0.004238,0.367775,0.215037,0.332881,0.117545,0.17925,0.498409,...,0.26886,0.169326,0.293812,0.407708,0.248496,0.162736,0.176733,0.194555,0.215864,0.183124,0.066061,0.152423,0.249621,0.054014,0.268747,0.133745,0.314511,0.175122,0.140422,0.279398,0.218278,0.156875,0.115979,0.201456,0.1163,0.155168,0.120762,0.083208,0.198613,0.149945,0.28866,0.228165,0.243741,0.171521,0.157897,0.239998,0.3082,0.338159,0.189674,0.157969


In [323]:
outputdf = pd.DataFrame()
finished = False
date = datetime.datetime.strptime('2021-10-31 04:00:00', "%Y-%m-%d %H:%M:%S") #The day our data collection starts
fillValue=0
while not finished:
  tempdf = GMEdf.loc[date:date+datetime.timedelta(days=1)].reset_index().drop('timestamp', axis=1)
  tempdf = tempdf.drop(df.index[144:tempdf.shape[0]]) #Sometimes due to the date grouping in SQL there is an extra datapoint, need to remove it here
  tempdf = tempdf.T.reset_index().drop('index', axis=1)
  if tempdf.shape[1] < 144:
    fillValue = tempdf.T.max().values[0]
    finished=True
  outputdf = outputdf.append(tempdf)
  date = date+datetime.timedelta(days=1)
  
outputdf.reset_index(inplace=True)
outputdf.drop('index', axis=1, inplace=True)
outputdf.fillna(method='ffill', inplace=True)
outputdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143
0,0.195662,0.048716,0.099136,0.143123,0.110418,0.112241,-0.157870,0.173691,0.072987,-0.000509,0.099020,0.107785,0.326317,0.217217,0.047507,0.261277,-0.014242,0.501520,-0.162287,0.190400,-0.072667,-0.012277,0.360258,0.097785,0.375767,0.389900,0.144700,0.109843,0.093240,-0.213060,-0.067009,0.179260,0.216360,0.461762,0.048140,0.203443,0.153127,0.516185,0.117462,0.439980,...,0.039685,0.290186,0.293854,0.245038,0.056535,0.174800,0.113331,0.240174,0.323974,0.273860,0.240011,0.315046,0.242068,0.166074,0.141642,0.328918,0.357443,0.171648,0.090839,0.268975,0.173252,0.036346,0.197431,0.185665,0.190853,0.136626,0.299795,-0.028488,0.246022,0.189874,0.305565,0.071652,0.322239,0.474333,0.069414,0.049210,0.348225,0.049525,0.129367,0.075843
1,0.114356,0.099109,0.175144,0.304779,0.345358,0.399038,-0.011435,0.222067,0.196858,0.172158,0.180473,0.061874,0.026794,0.085606,0.382719,0.298168,0.099157,0.201164,0.198460,0.112562,0.363038,0.255517,0.174993,0.232100,0.200550,0.408267,0.662525,0.523527,0.150310,0.260815,0.178775,0.293928,0.506040,-0.004238,0.367775,0.215037,0.332881,0.117545,0.179250,0.498409,...,0.268860,0.169326,0.293812,0.407708,0.248496,0.162736,0.176733,0.194555,0.215864,0.183124,0.066061,0.152423,0.249621,0.054014,0.268747,0.133745,0.314511,0.175122,0.140422,0.279398,0.218278,0.156875,0.115979,0.201456,0.116300,0.155168,0.120762,0.083208,0.198613,0.149945,0.288660,0.228165,0.243741,0.171521,0.157897,0.239998,0.308200,0.338159,0.189674,0.157969
2,0.187685,0.138213,0.186307,0.235348,0.161379,0.181908,0.150029,0.118785,0.511282,0.160296,0.400111,0.248338,0.233004,0.330061,0.317057,0.092212,0.081692,0.211875,0.452859,0.147500,0.105083,0.151983,0.214392,0.110428,0.428225,0.041023,0.095494,0.187825,0.335453,0.294286,0.063453,0.045960,0.280993,0.252200,0.344647,0.187940,0.145453,0.176595,0.051805,0.062259,...,0.070876,0.011805,0.081229,0.051119,0.061596,0.187347,0.152166,0.174932,0.075584,0.116614,0.114098,0.238307,0.096879,0.130007,0.122375,0.046927,0.198093,0.236295,-0.019538,0.042575,0.072259,0.044338,0.172055,0.267061,0.140629,0.213118,0.015851,0.109838,0.276535,0.190578,0.175912,0.122884,0.203064,0.124311,0.062234,0.077996,0.185966,0.078602,0.136085,0.106877
3,0.068522,0.186555,0.081814,0.093659,0.022609,-0.014108,0.181455,0.216154,0.070626,0.233096,0.125832,0.030072,0.085658,0.159439,-0.077925,0.071406,-0.115433,0.371705,0.307287,0.044187,0.136755,0.283030,0.154412,0.145356,0.231270,-0.035071,0.078087,0.177200,0.084893,0.183188,0.174625,0.083625,0.248091,0.205838,-0.014982,0.210979,0.014564,0.235914,0.273545,0.140267,...,0.208044,0.152698,0.113920,0.122111,0.144444,0.225967,0.174802,0.233975,0.214344,0.044173,0.284224,0.122244,0.200778,-0.018910,0.077294,0.248853,0.213412,0.183285,0.187576,0.072261,0.127030,0.150569,0.212147,0.243532,0.199283,0.321108,0.034671,0.220705,0.166280,0.286357,0.191034,0.160351,0.220011,0.395985,0.197184,0.196900,0.246206,0.096631,0.147287,0.259056
4,0.171625,0.314512,0.211357,0.297655,0.265775,0.131890,0.251644,0.118250,0.014907,0.299977,0.416467,0.091632,0.387107,0.257456,0.167538,0.047457,0.374329,0.162700,0.055927,0.231488,0.150500,-0.091320,0.443275,0.142143,0.029973,0.215729,0.094570,0.299571,-0.059156,0.327770,0.083176,0.297450,0.062777,0.041884,0.191373,0.353228,0.238412,0.256547,0.429367,0.079982,...,0.229302,0.270419,0.261538,0.156767,0.215295,0.200880,0.180248,0.030414,0.276340,0.229118,0.371128,0.077086,0.193272,0.077794,0.110809,0.209098,-0.000229,0.294863,0.349709,0.306748,0.180559,0.206708,0.041652,0.227257,0.222236,0.348781,0.068376,0.118897,0.239528,0.262775,0.231525,0.408185,0.109948,0.188383,0.044333,0.209000,-0.167467,0.149462,0.098788,0.258046
5,0.135545,0.125311,0.262244,0.001815,0.187194,-0.034071,0.391860,0.09154

Next get the prices for each day

In [324]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [325]:
GMEpricesDf = pd.read_csv('/content/drive/MyDrive/Data/GME.csv')
GMEpricesDf['Date'] = pd.to_datetime(GMEpricesDf['Date'])
GMEpricesDf.set_index('Date', inplace=True)
GMEpricesDf = GMEpricesDf.resample('D').pad()

In [328]:
outputdf['Close'] = np.nan
outputdf['PrevClose'] = np.nan
curdate = first_data_collection_date
for i in range(outputdf.shape[0]):
  outputdf.loc[i, 'PrevClose'] = GMEpricesDf.loc[curdate.strftime("%Y-%m-%d")]['Close']
  curdate=first_data_collection_date+datetime.timedelta(days=i)
  outputdf.loc[i, 'Close'] = GMEpricesDf.loc[curdate.strftime("%Y-%m-%d")]['Close']
outputdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,Close,PrevClose
0,0.195662,0.048716,0.099136,0.143123,0.110418,0.112241,-0.157870,0.173691,0.072987,-0.000509,0.099020,0.107785,0.326317,0.217217,0.047507,0.261277,-0.014242,0.501520,-0.162287,0.190400,-0.072667,-0.012277,0.360258,0.097785,0.375767,0.389900,0.144700,0.109843,0.093240,-0.213060,-0.067009,0.179260,0.216360,0.461762,0.048140,0.203443,0.153127,0.516185,0.117462,0.439980,...,0.293854,0.245038,0.056535,0.174800,0.113331,0.240174,0.323974,0.273860,0.240011,0.315046,0.242068,0.166074,0.141642,0.328918,0.357443,0.171648,0.090839,0.268975,0.173252,0.036346,0.197431,0.185665,0.190853,0.136626,0.299795,-0.028488,0.246022,0.189874,0.305565,0.071652,0.322239,0.474333,0.069414,0.049210,0.348225,0.049525,0.129367,0.075843,183.509995,183.509995
1,0.114356,0.099109,0.175144,0.304779,0.345358,0.399038,-0.011435,0.222067,0.196858,0.172158,0.180473,0.061874,0.026794,0.085606,0.382719,0.298168,0.099157,0.201164,0.198460,0.112562,0.363038,0.255517,0.174993,0.232100,0.200550,0.408267,0.662525,0.523527,0.150310,0.260815,0.178775,0.293928,0.506040,-0.004238,0.367775,0.215037,0.332881,0.117545,0.179250,0.498409,...,0.293812,0.407708,0.248496,0.162736,0.176733,0.194555,0.215864,0.183124,0.066061,0.152423,0.249621,0.054014,0.268747,0.133745,0.314511,0.175122,0.140422,0.279398,0.218278,0.156875,0.115979,0.201456,0.116300,0.155168,0.120762,0.083208,0.198613,0.149945,0.288660,0.228165,0.243741,0.171521,0.157897,0.239998,0.308200,0.338159,0.189674,0.157969,200.089996,183.509995
2,0.187685,0.138213,0.186307,0.235348,0.161379,0.181908,0.150029,0.118785,0.511282,0.160296,0.400111,0.248338,0.233004,0.330061,0.317057,0.092212,0.081692,0.211875,0.452859,0.147500,0.105083,0.151983,0.214392,0.110428,0.428225,0.041023,0.095494,0.187825,0.335453,0.294286,0.063453,0.045960,0.280993,0.252200,0.344647,0.187940,0.145453,0.176595,0.051805,0.062259,...,0.081229,0.051119,0.061596,0.187347,0.152166,0.174932,0.075584,0.116614,0.114098,0.238307,0.096879,0.130007,0.122375,0.046927,0.198093,0.236295,-0.019538,0.042575,0.072259,0.044338,0.172055,0.267061,0.140629,0.213118,0.015851,0.109838,0.276535,0.190578,0.175912,0.122884,0.203064,0.124311,0.062234,0.077996,0.185966,0.078602,0.136085,0.106877,206.990005,200.089996
3,0.068522,0.186555,0.081814,0.093659,0.022609,-0.014108,0.181455,0.216154,0.070626,0.233096,0.125832,0.030072,0.085658,0.159439,-0.077925,0.071406,-0.115433,0.371705,0.307287,0.044187,0.136755,0.283030,0.154412,0.145356,0.231270,-0.035071,0.078087,0.177200,0.084893,0.183188,0.174625,0.083625,0.248091,0.205838,-0.014982,0.210979,0.014564,0.235914,0.273545,0.140267,...,0.113920,0.122111,0.144444,0.225967,0.174802,0.233975,0.214344,0.044173,0.284224,0.122244,0.200778,-0.018910,0.077294,0.248853,0.213412,0.183285,0.187576,0.072261,0.127030,0.150569,0.212147,0.243532,0.199283,0.321108,0.034671,0.220705,0.166280,0.286357,0.191034,0.160351,0.220011,0.395985,0.197184,0.196900,0.246206,0.096631,0.147287,0.259056,218.330002,206.990005
4,0.171625,0.314512,0.211357,0.297655,0.265775,0.131890,0.251644,0.118250,0.014907,0.299977,0.416467,0.091632,0.387107,0.257456,0.167538,0.047457,0.374329,0.162700,0.055927,0.231488,0.150500,-0.091320,0.443275,0.142143,0.029973,0.215729,0.094570,0.299571,-0.059156,0.327770,0.083176,0.297450,0.062777,0.041884,0.191373,0.353228,0.238412,0.256547,0.429367,0.079982,...,0.261538,0.156767,0.215295,0.200880,0.180248,0.030414,0.276340,0.229118,0.371128,0.077086,0.193272,0.077794,0.110809,0.209098,-0.000229,0.294863,0.349709,0.306748,0.180559,0.206708,0.041652,0.227257,0.222236,0.348781,0.068376,0.118897,0.239528,0.262775,0.231525,0.408185,0.109948,0.188383,0.044333,0.209000,-0.167467,0.149462,0.098788,0.258046,217.839996,218.330002
5,0.135545,0.125311,0.262244,0.001815,0.18719

Basic neural network

In [329]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import regularizers

In [330]:
target = outputdf.pop('Close')

In [331]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(outputdf)

In [345]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)), 
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanAbsolutePercentageError(reduction="auto", name="MeanAbsolutePercentageError"))
  return model

In [367]:
model = get_basic_model()
model.fit(outputdf[:-1], target[:-1], epochs=500, batch_size=5, validation_data=[outputdf[-1:], target[-1:]])

Epoch 1/500
4/4 [==============================] - 2s 106ms/step - loss: 105.5167 - val_loss: 105.1643
Epoch 2/500
4/4 [==============================] - 0s 10ms/step - loss: 105.0433 - val_loss: 104.7049
Epoch 3/500
4/4 [==============================] - 0s 10ms/step - loss: 104.5871 - val_loss: 104.2673
Epoch 4/500
4/4 [==============================] - 0s 11ms/step - loss: 104.2075 - val_loss: 103.9384
Epoch 5/500
4/4 [==============================] - 0s 9ms/step - loss: 103.8764 - val_loss: 103.5912
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 103.5211 - val_loss: 103.2481
Epoch 7/500
4/4 [==============================] - 0s 10ms/step - loss: 103.1343 - val_loss: 102.8477
Epoch 8/500
4/4 [==============================] - 0s 12ms/step - loss: 102.7399 - val_loss: 102.4834
Epoch 9/500
4/4 [==============================] - 0s 10ms/step - loss: 102.4108 - val_loss: 102.1725
Epoch 10/500
4/4 [==============================] - 0s 10ms/step - loss: 102.0740 

In [371]:
print("Predicted value: %f, True value: %f"%(model.predict(outputdf)[-1:][0][0], target[-1:].values[0]))
print("Percent error on unseen datapoint: %f"%(((target[-1:].values[0] - model.predict(outputdf)[-1:][0][0]) / target[-1:].values[0])*100))

Predicted value: 210.048935, True value: 210.850006
Percent error on unseen datapoint: 0.379925
